# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [28]:
llm = OpenAI(temperature=0)

In [29]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [30]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [31]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [32]:
import os
from langchain_community.document_loaders import WebBaseLoader

os.environ["USER_AGENT"] = "MyVectorStoreBot/1.0 (+https://github.com/my-project)"
loader = WebBaseLoader(
    "https://…",
    header_template={"User-Agent": os.environ["USER_AGENT"]},
)


In [33]:
from langchain_community.document_loaders import WebBaseLoader

In [34]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [35]:
loader

In [39]:
raw_docs = loader.load()    

In [40]:
raw_docs

[Document(metadata={'source': 'https://beta.ruff.rs/docs/faq/', 'title': 'FAQ | Ruff', 'description': 'An extremely fast Python linter and code formatter, written in Rust.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\nFAQ | Ruff\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Ruff\n          \n\n\n\n            \n              FAQ\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    ruff\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Ruff\n  \n\n\n\n\n\n\n    ruff\n  \n\n\n\n\n\n\n    \n  \n    Overview\n  \n\n    \n  \n\n\n\n\n\n    \n  \n    Tutorial\n  \n\n    \n  \n\n\n\n\n\n    \n  \n    Installing Ruff\n  \n\n    \n  \n\n\n\n\n\n    \n  \n    The Ruff Linter\n  \n\n    \n  \n\n\n\n\n\n    \n  \n    The Ruff Formatter\n  \n\n    \n  \n\n\n\n\n\n\n    \n  \n    Editors\

In [41]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [46]:
import pandas as pd
from bs4 import BeautifulSoup
from langchain.schema import Document

# 1. Extract plain text from HTML
plain_texts = []
for doc in raw_docs:
    soup = BeautifulSoup(doc.page_content, "html.parser")
    text = soup.get_text(separator="\n")        # preserve paragraph breaks
    plain_texts.append(text)

# 2. Split into paragraphs and build DataFrame
paras = []
for text in plain_texts:
    for p in text.split("\n\n"):                # split on double-newline
        p = p.strip()
        if p:
            paras.append(p)

df = pd.DataFrame({"text": paras})

# 3. Normalize whitespace (collapse internal runs of spaces/newlines) 
#    and drop any empty rows just in case:
df["text"] = df["text"].str.replace(r"\s+", " ", regex=True).str.strip()
df = df[df["text"] != ""]

print(df.shape)  # now (n_paragraphs, 1)
# print(df())
df

(108, 1)


,text
0,FAQ | Ruff
1,Skip to content
2,Ruff
3,FAQ
4,Initializing search
...,...
103,"Ruff doesn't currently support INI files, like..."
104,"Note Prior to v0.5.0, Ruff would read user-spe..."
105,"For more, see the etcetera crate. Ruff tried t..."
106,Back to top


In [ ]:
# from langchain.schema import Document
# from langchain_text_splitters import CharacterTextSplitter

# # wrap cleaned paragraphs back into Documents
# clean_docs = [Document(page_content=t) for t in df["text"]]

# splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = splitter.split_documents(clean_docs)

# now build your vector store on `texts`


In [ ]:
# texts

[Document(metadata={}, page_content='FAQ | Ruff'),
 Document(metadata={}, page_content='Skip to content'),
 Document(metadata={}, page_content='Ruff'),
 Document(metadata={}, page_content='FAQ'),
 Document(metadata={}, page_content='Initializing search'),
 Document(metadata={}, page_content='ruff'),
 Document(metadata={}, page_content='Ruff'),
 Document(metadata={}, page_content='ruff'),
 Document(metadata={}, page_content='Overview'),
 Document(metadata={}, page_content='Tutorial'),
 Document(metadata={}, page_content='Installing Ruff'),
 Document(metadata={}, page_content='The Ruff Linter'),
 Document(metadata={}, page_content='The Ruff Formatter'),
 Document(metadata={}, page_content='Editors'),
 Document(metadata={}, page_content='Editors'),
 Document(metadata={}, page_content='Editor Integration'),
 Document(metadata={}, page_content='Setup'),
 Document(metadata={}, page_content='Features'),
 Document(metadata={}, page_content='Settings'),
 Document(metadata={}, page_content='Migr

In [50]:


docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [51]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [52]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [53]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\happy\AppData\Local\Temp\ipykernel_41472\1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [54]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I should use the Ruff QA System to get more information about Ketanji Brown Jackson.
Action: Ruff QA System
Action Input: "Who is Ketanji Brown Jackson?"
Observation:  I don't know.
Thought: I should use the State of Union QA System again to get more information about Ketanji Brown Jackson.
Action: State of Union QA System
Action Input: "Who is Ketanji Brown Jackson?"
Observation:  Ketanji Brown Jackson is a Circuit Court of Appeals Judge who was nominated by President Biden to serve on th

{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': 'Ketanji Brown Jackson is a Circuit Court of Appeals Judge who was nominated by President Biden to serve on the United States Supreme Court.'}

In [55]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.
Thought: Now that I know the differences, I can make an informed decision.
Final Answer: It ultimately depends on your specific needs and preferences, but some potential reasons to choose ruff over flake8 could include its larger rule set and automatic fixing capabilities.

> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': 'It ultimately depends on your specific needs and preferences, but some potential reasons to choose ruff over flake8 could include its larger rule set and automatic fixing capabilities.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [56]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [57]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [58]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [59]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': " Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [60]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [61]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [62]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: I should use the State of Union QA System to check if the president mentioned this tool.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation:  No, the president did not mention any specific tool in the State of the Union address.
Thought: I now know the final answer.
Final Answer: The final answer is that the president did not mention any specific tool in the State of the Union address.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'The final answer is that the president did not mention any specific tool in the State of the Union address.'}